**Import Required Packages** 

**Listing the files in input**

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

# Any results you write to the current directory are saved as output.

**Initialize Test and Training Set**

In [ ]:
train_set = pd.read_csv("../input/train.csv")
test_set = pd.read_csv("../input/test.csv")
train_set.head()
#test_final = pd.DataFrame
#test_final


In [ ]:
train_set.isnull().sum()
test_set.isnull().sum()

In [ ]:
train_set

In [ ]:
def get_title(name):
	title_search = re.search(' ([A-Za-z]+)\.', name)
	# If the title exists, extract and return it.
	if title_search:
		return title_search.group(1)
	return ""

train_set['Title'] = train_set['Name'].apply(get_title)
test_set['Title'] = test_set['Name'].apply(get_title)

print(pd.crosstab(train_set['Title'], train_set['Sex']))


train_set['Title'] = train_set['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train_set['Title'] = train_set['Title'].replace('Mlle', 'Miss')
train_set['Title'] = train_set['Title'].replace('Ms', 'Miss')
train_set['Title'] = train_set['Title'].replace('Mme', 'Mrs')
test_set['Title'] = test_set['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test_set['Title'] = test_set['Title'].replace('Mlle', 'Miss')
test_set['Title'] = test_set['Title'].replace('Ms', 'Miss')
test_set['Title'] = test_set['Title'].replace('Mme', 'Mrs')

print (train_set[['Title', 'Survived']].groupby(['Title'], as_index=False).mean())

In [ ]:
#train_set.loc[train_set["Age"]==pd.np.nan, "Age" ] = 100
#train_set["Age"].replace(np.NaN, train_set["Age"].mean())
train_set["Age"] = np.where(train_set["Age"].isnull(), train_set["Age"].mean(), train_set["Age"] )
train_set["Fare"] = np.where(train_set["Fare"].isnull(), train_set["Fare"].mean(), train_set["Fare"] )
train_set["Cabin"] = np.where(train_set["Cabin"].isnull(), "U", train_set["Cabin"] )
train_set["Embarked"] = np.where(train_set["Embarked"].isnull(), "U", train_set["Embarked"] )
train_set["Pclass"] = np.where(train_set["Pclass"].isnull(), 0, train_set["Pclass"] )
train_set["SibSp"] = np.where(train_set["SibSp"].isnull(), 0, train_set["SibSp"] )
train_set["Parch"] = np.where(train_set["Parch"].isnull(), 0, train_set["Parch"] )


test_set["Age"] = np.where(test_set["Age"].isnull(), test_set["Age"].mean(), test_set["Age"] )
test_set["Fare"] = np.where(test_set["Fare"].isnull(), test_set["Fare"].mean(), test_set["Fare"] )
test_set["Cabin"] = np.where(test_set["Cabin"].isnull(), "U", test_set["Cabin"] )
test_set["Embarked"] = np.where(test_set["Embarked"].isnull(), "U", test_set["Embarked"] )
test_set["Pclass"] = np.where(test_set["Pclass"].isnull(), 0, test_set["Pclass"] )
test_set["SibSp"] = np.where(test_set["SibSp"].isnull(), 0, test_set["SibSp"] )
test_set["Parch"] = np.where(test_set["Parch"].isnull(), 0, test_set["Parch"] )


test_set.head()

In [ ]:
train_set.describe()

In [ ]:
train_set.corr().abs()

**Binning Columns:(Quantile based adaptive binning)**

**The data is right skewed So going for quantile based binning** 

In [ ]:
quantile_list = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
age_quantiles = train_set["Age"].quantile(quantile_list)
fare_quantiles = train_set["Fare"].quantile(quantile_list)

age_quantiles

**Applying Fixed-width binning and applying Quantile to see the skewness**

In [ ]:

#fig, ax = plt.subplots()
test_set["Age"].hist(bins=10, color='#A9C5D3', edgecolor='black', grid=False)
#train_set["Fare"].hist(bins=10, color='#A9C5D3', edgecolor='black', grid=False)
for quantile in age_quantiles:
    qvl = plt.axvline(quantile, color='g')


In [ ]:
quantile_labels = ["1", "2", "3", "4", "5"]
family_labels = ["1","2"]
train_set['Age_quantile_range'] = pd.qcut(train_set['Age'], q=quantile_list)
train_set['Fare_quantile_range'] = pd.qcut(train_set['Fare'], q=quantile_list)
train_set['Age_quantile_range'] = pd.qcut(train_set['Age'], q=quantile_list, labels=quantile_labels)
train_set['Fare_quantile_range'] = pd.qcut(train_set['Fare'], q=quantile_list, labels=quantile_labels)
test_set['Fare_quantile_range'] = pd.qcut(test_set['Fare'], q=quantile_list, labels=quantile_labels)
test_set['Age_quantile_range'] = pd.qcut(test_set['Age'], q=quantile_list, labels=quantile_labels)

test_set


**Doing split of train and validation set**

In [ ]:
labels = train_set["Survived"] 
train_final = train_set.drop(["Survived", "Age", "Name", "Ticket", "PassengerId","Fare"], axis=1)
test_final = test_set.drop(["Age", "Name", "Ticket", "PassengerId","Fare"], axis=1)
train_final = pd.get_dummies(train_final)
test_final = pd.get_dummies(test_final)
test_final_1 = pd.DataFrame(columns=train_final.columns)
print(str(train_final.shape)+"===>"+str(test_final_1.shape))
for column in train_final.columns:
    if column not in test_final.columns:
        test_final_1[column] = 0
    else:
        test_final_1[column] = test_final[column]
    
print(str(train_final.shape)+"===>"+str(test_final_1.shape))

print(str(train_final.shape)+"===>"+str(test_final_1.shape))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(train_final, labels, test_size=0.20, random_state=19)
print(str(X_train.shape)+"===>"+str(X_test.shape))
X_train

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(train_final, labels, test_size=0.20, random_state=50)
print(str(X_train.shape)+"===>"+str(X_test.shape))
X_train
rf1 = RandomForestClassifier(max_depth=10, n_estimators = 40, random_state = 100)
rf1.fit(X_train,Y_train)
y_pred = rf1.predict(X_test)
y_pred
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, y_pred))  

Y_test.values
Y_train
y_pred

print(confusion_matrix(Y_test,y_pred))  
print(classification_report(Y_test,y_pred))  
print(accuracy_score(Y_test, y_pred)) 

In [ ]:
rf = RandomForestClassifier(max_depth=10,n_estimators = 40, random_state = 100)
rf.fit(train_final,labels)
predictions = rf.predict(test_final_1)
predictions
#from sklearn import metrics
#from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
#print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, y_pred))  

#Y_test.values
#Y_train
#y_pred

#print(confusion_matrix(Y_test,y_pred))  
#print(classification_report(Y_test,y_pred))  
#print(accuracy_score(Y_test, y_pred)) 
submission = pd.DataFrame({'PassengerId':test_set['PassengerId'],'Survived':predictions})
submission
#predictions


filename = 'Titanic Predictions 1.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)
